In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from typing_extensions import List, TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: list
    answer: str
    
graph_builder = StateGraph(AgentState)  

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", streaming=True)
small_llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal

class Route(BaseModel):
   target: Literal['income_tax', 'llm', 'real_estate_tax'] = Field(
        description="The target for the query to answer"
   )
   
router_system_prompt = """
You are an expert at routing a user's question to 'income_tax', 'llm', or 'real_estate_tax'.
'income_tax' contains information about income tax up to December 2024.
'real_estate_tax' contains information about real estate tax up to December 2024.
if you think the question is not related to either 'income_tax' or 'real_estate_tax';
you can route it to 'llm'."""   

router_prompt = ChatPromptTemplate.from_messages([
    ("system", router_system_prompt),
    ("user", "{query}"),
])
                                                 
structured_router_llm = small_llm.with_structured_output(Route)

def router(state: AgentState) -> Literal['income_tax', 'llm', 'real_estate_tax']:
   """
   주어진 state에서 쿼리를 기반으로 적절한 경로를 결정합니다.
    Args:
         state (AgentState): 에이전트의 현재 상태를 나타내는 딕셔너리.
    
    Returns:
         Literal['income_tax', 'llm', 'real_estate_tax']: 쿼리에 따라 선택된 경로를 반환합니다.
   """
   
   query = state['query']
   router_chain = router_prompt | structured_router_llm
   route = router_chain.invoke({"query": query})
   
   return route.target    

In [19]:
from langchain_core.output_parsers import StrOutputParser

def call_llm(state: AgentState) -> AgentState:
    """
    주어진 state에서 쿼리를 기반으로 LLM을 호출하여 답변을 생성합니다.
    
    Args:
        state (AgentState): 에이전트의 현재 상태를 나타내는 딕셔너리.
    
    Returns:
        AgentState: 'answer' 키에 LLM의 응답을 포함하는 AgentState를 반환합니다.
    """
   
    query = state['query']
    llm_chain = small_llm | StrOutputParser()
    answer = llm_chain.invoke(query)
    
    return {'answer': answer}

In [20]:
from income_tax_agent import graph as income_tax_agent
from real_estate_tax_graph import graph as real_estate_tax_agent

graph_builder.add_node('income_tax', income_tax_agent)
graph_builder.add_node('real_estate_tax', real_estate_tax_agent)
graph_builder.add_node('llm', call_llm)

c:\workspace\uv_work\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\workspace\uv_work\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\workspace\uv_work\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\workspace\uv_work\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [21]:
from langgraph.graph import START, END

graph_builder.add_conditional_edges(
    START,
    router,
    {
        'income_tax': 'income_tax',
        'real_estate_tax': 'real_estate_tax',
        'llm': 'llm'
    }
)

graph_builder.add_edge('income_tax', END)
graph_builder.add_edge('real_estate_tax', END)
graph_builder.add_edge('llm', END)

In [22]:
graph = graph_builder.compile()

In [23]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [24]:
initial_state = {'query': '연봉 5천만원 거주자의 소득세는 얼마인가요?'}
graph.invoke(initial_state)

거짓말:  not hallucinated
유용성: helpful


{'query': '연봉 5천만원 거주자의 소득세는 얼마인가요?',
 'context': [Document(metadata={'source': './tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 

In [25]:
initial_state = {'query': '집 15억은 세금이 얼마인가요?'}
graph.invoke(initial_state)

{'query': '집 15억은 세금이 얼마인가요?',
 'answer': '주어진 정보를 바탕으로 1세대 1주택자인 경우와 1세대 2주택자인 경우의 종합부동산세를 각각 계산해 보겠습니다.\n\n### 1. 1세대 1주택자인 경우\n- 과세표준: 3억 원\n- 주택 수: 1\n- 세율: 1천분의 5\n\n세액 = 과세표준 × 세율 = 3억 원 × 0.0005 = 150만 원\n\n### 2. 1세대 2주택자인 경우\n- 과세표준: 6억 원\n- 주택 수: 2\n- 세율: 6억 원 이하이므로, 3억 원 초과 6억 원 이하 구간에 해당\n  - 기본 세액: 150만 원\n  - 추가 세율: 1천분의 7\n\n세액 = 기본 세액 + (초과 금액 × 추가 세율)\n     = 150만 원 + ((6억 원 - 3억 원) × 0.0007)\n     = 150만 원 + (3억 원 × 0.0007)\n     = 150만 원 + 210만 원\n     = 360만 원\n\n따라서, 주택의 공시가격이 15억 원인 경우, 종합부동산세는 1세대 1주택자일 때 150만 원, 1세대 2주택자일 때 360만 원으로 계산됩니다.'}

In [26]:
initial_state = {'query': '떡볶이는 어디가 맛있나요?'}
graph.invoke(initial_state)

{'query': '떡볶이는 어디가 맛있나요?',
 'answer': '떡볶이는 한국에서 매우 인기 있는 길거리 음식으로, 여러 지역에서 다양한 맛을 즐길 수 있습니다. 특히 서울의 명동, 홍대, 강남 등에서는 유명한 떡볶이 가게들이 많습니다. \n\n1. **명동**: 명동 떡볶이는 다양한 종류의 떡볶이를 맛볼 수 있는 곳으로, 특히 치즈 떡볶이가 인기가 많습니다.\n2. **홍대**: 홍대 지역은 젊은 층이 많이 찾는 곳으로, 독특한 맛과 다양한 토핑을 제공하는 가게들이 많습니다.\n3. **신림**: 신림에서도 맛있는 떡볶이를 찾을 수 있으며, 특히 가성비 좋은 가게들이 많습니다.\n\n각 지역마다 특색 있는 떡볶이를 제공하니, 여러 곳을 방문해보는 것도 좋은 방법입니다!'}